In [4]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from IPython.display import clear_output, display
from FunBuildBatch import *
pd.options.display.float_format = '{:.0f}'.format

In [5]:
%%time
DicConfig = LoadJsonFile(os.path.join(os.getcwd(),"config.json"),"somejson")
GlobalDicDeplier(DicConfig)
tweetslist,errors = ReadTweetsToList(Root,FolderProject,TweetsFilename)

In [6]:
%%time
f = "%a %b %d %H:%M:%S +0000 %Y"
df = pd.DataFrame(tweetslist)
df.TWEETTIMESTAMP = pd.to_datetime(df.TWEETTIMESTAMP,format = f, errors='ignore')
df["TWEETUNIXEPOCH"] = df.TWEETTIMESTAMP.map(lambda a : a.value // 10**9)
df.TWEETUNIXEPOCH = df.TWEETUNIXEPOCH + 3600

CPU times: user 31.8 s, sys: 673 ms, total: 32.5 s
Wall time: 32.5 s


In [7]:
%%time
df = df.replace({'True':True,'False':False})
df.USERID = pd.to_numeric(df.USERID,errors="coerce")
df.USERFOLLOWERS = pd.to_numeric(df.USERFOLLOWERS,errors="coerce")
df.TWEETID = pd.to_numeric(df.TWEETID,errors="coerce")
df.AUTHORID = pd.to_numeric(df.AUTHORID,errors="coerce")
df.AUTHORFOLLOWERS = pd.to_numeric(df.AUTHORFOLLOWERS,errors="coerce")
df.AUTHORTWEETID = pd.to_numeric(df.AUTHORTWEETID,errors="coerce")
df.TWEETUNIXEPOCH = pd.to_numeric(df.TWEETUNIXEPOCH,errors="coerce")

CPU times: user 57.2 s, sys: 3.4 s, total: 1min
Wall time: 58.8 s


In [8]:
%%time
dfrt = df.copy()[df.ISRETWEET]
dfrt.dropna(inplace=True)
dfrt.AUTHORTWEETTIMESTAMP = pd.to_datetime(dfrt.AUTHORTWEETTIMESTAMP,format = f, errors='ignore')
dfrt["AUTHORTWEETUNIXEPOCH"] = dfrt.AUTHORTWEETTIMESTAMP.map(lambda a : a.value // 10**9)
dfrt.AUTHORTWEETUNIXEPOCH = dfrt.AUTHORTWEETUNIXEPOCH + 3600


dfsimple = df.copy()[~df.ISRETWEET]
vartoremove = [ 'AUTHORNAME',
 'AUTHORFNAME',
 'AUTHORID',
 'AUTHORVERIFIED',
 'AUTHORDESCRIPTION',
 'AUTHORFOLLOWERS',
 'AUTHORTWEETID',
 'AUTHORTWEETCONTENT',
 'AUTHORTWEETTIMESTAMP']
dfsimple.drop(columns=vartoremove,inplace=True)
dfsimple.dropna(inplace=True)

CPU times: user 24.5 s, sys: 785 ms, total: 25.3 s
Wall time: 24.3 s


# Table Influenceur

In [9]:
keepvar = ["AUTHORID","AUTHORNAME","AUTHORFNAME","AUTHORDESCRIPTION","AUTHORFOLLOWERS"]
TableInf = dfrt.copy()
TableInf = TableInf[keepvar]
TableInf = TableInf.groupby("AUTHORID").first().reset_index()

# Table RT

In [10]:
keepvar = ["TWEETID","AUTHORTWEETID","USERID","AUTHORID","TWEETUNIXEPOCH"]
TableRT = dfrt.copy()
TableRT = TableRT[keepvar]

# Table Tweet Repris
...

In [10]:
%%time
PickleDump(os.path.join(Root,FolderProject,"BatchTweetSimple.pkl"),dfsimple)
PickleDump(os.path.join(Root,FolderProject,"BatchRetweet.pkl"),dfrt)